## Problem Statement:

Create a simple language n-grams model that predicts the next word in a sentence

### Data collection and pre-processing

In [1]:
import nltk
from nltk.corpus import gutenberg, reuters
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
import math
import sys

In [2]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
len(gutenberg.sents('whitman-leaves.txt'))

4250

## Example sentences 

In [4]:
gutenberg.sents('whitman-leaves.txt')[15:25]

[['Here',
  'are',
  'our',
  'thoughts',
  ',',
  'voyagers',
  "'",
  'thoughts',
  ',',
  'Here',
  'not',
  'the',
  'land',
  ',',
  'firm',
  'land',
  ',',
  'alone',
  'appears',
  ',',
  'may',
  'then',
  'by',
  'them',
  'be',
  'said',
  ',',
  'The',
  'sky',
  'o',
  "'",
  'erarches',
  'here',
  ',',
  'we',
  'feel',
  'the',
  'undulating',
  'deck',
  'beneath',
  'our',
  'feet',
  ',',
  'We',
  'feel',
  'the',
  'long',
  'pulsation',
  ',',
  'ebb',
  'and',
  'flow',
  'of',
  'endless',
  'motion',
  ',',
  'The',
  'tones',
  'of',
  'unseen',
  'mystery',
  ',',
  'the',
  'vague',
  'and',
  'vast',
  'suggestions',
  'of',
  'the',
  'briny',
  'world',
  ',',
  'the',
  'liquid',
  '-',
  'flowing',
  'syllables',
  ',',
  'The',
  'perfume',
  ',',
  'the',
  'faint',
  'creaking',
  'of',
  'the',
  'cordage',
  ',',
  'the',
  'melancholy',
  'rhythm',
  ',',
  'The',
  'boundless',
  'vista',
  'and',
  'the',
  'horizon',
  'far',
  'and',
  'dim',


In [5]:
text = gutenberg.raw('austen-emma.txt')

## Cleaning the text

In [6]:
def clean_text(text):
    
    #Changing to lowerecase
    text = text.lower()
    # Removing URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters
    text = re.sub(r'\W', ' ', text)
    # Replace numbers with nothing
    text = re.sub(r'\d+', '', text)
    # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text

In [7]:
cleaned_text = clean_text(text)

## Tokenization

In [8]:
tokens = word_tokenize(cleaned_text)

In [9]:
tokens[:20]

['emma',
 'by',
 'jane',
 'austen',
 'volume',
 'chapter',
 'emma',
 'woodhouse',
 'handsome',
 'clever',
 'and',
 'rich',
 'with',
 'comfortable',
 'home',
 'and',
 'happy',
 'disposition',
 'seemed',
 'to']

## n-grams

In [10]:
def generate_ngrams(tokens, n):
    n_grams = ngrams(tokens, n)
    n_grams_freq = Counter(n_grams)
    return n_grams_freq

In [11]:
bigrams = generate_ngrams(tokens, 2)
trigrams = generate_ngrams(tokens, 3)

In [12]:
#10 most common bigrams and trigrams
print("Most common bigrams:", bigrams.most_common(10))
print("Most common trigrams:", trigrams.most_common(10))

Most common bigrams: [(('to', 'be'), 607), (('of', 'the'), 566), (('it', 'was'), 448), (('in', 'the'), 446), (('she', 'had'), 332), (('she', 'was'), 328), (('had', 'been'), 309), (('mr', 'knightley'), 300), (('it', 'is'), 299), (('could', 'not'), 278)]
Most common trigrams: [(('she', 'could', 'not'), 72), (('it', 'would', 'be'), 63), (('would', 'have', 'been'), 60), (('do', 'not', 'know'), 55), (('it', 'was', 'not'), 55), (('she', 'had', 'been'), 53), (('mr', 'frank', 'churchill'), 50), (('in', 'the', 'world'), 49), (('could', 'not', 'be'), 45), (('it', 'will', 'be'), 42)]


## Probability of a word given (n-1) gram

In [13]:
def calculate_probability(tokens, sequence, token, n):
    
    #n-grams
    ngram_freq = generate_ngrams(tokens,n)
    n1gram_freq = generate_ngrams(tokens,n-1)
        
    seq = tuple(sequence)
    seq_freq = n1gram_freq.get(seq, 0)
        
    # Get the frequency of the n-gram (context followed by the word)
    n_gram = seq + (token,)
    n_gram_freq = ngram_freq.get(n_gram, 0)
    
    # Calculate the probability
    if seq_freq == 0:
        return 0
    else:
        return n_gram_freq / seq_freq

In [14]:
sequence = ['mr','frank']
token = 'churchill'

probability = calculate_probability(tokens, sequence, token, 3)
print(f'Probability of {token} after {sequence} is {probability}')

Probability of churchill after ['mr', 'frank'] is 1.0


## Predict next word given a set of words

In [15]:
def predict_next_word(tokens,sequence,n):
    
    ngram_freq = generate_ngrams(tokens,n)
    n1gram_freq = generate_ngrams(tokens,n-1)
    
    possible_words = set(tokens)
    next_possible_word = None
    max_prob = 0
    
    for word in possible_words:
        prob = calculate_probability(tokens, sequence, word, 3)
        if prob > max_prob:
            max_prob = prob
            next_possible_word = word
    return next_possible_word

In [16]:
n = 3
context = ['churchill', 'was']
predicted_word = predict_next_word(tokens, context, 3)
print(f"The predicted next word following the context '{context}' is '{predicted_word}'")


The predicted next word following the context '['churchill', 'was']' is 'looking'


In [17]:
print(predicted_word)

looking


In [18]:
def generate_sentence(tokens, prefix, n, length):
    n_grams_freq = generate_ngrams(tokens, n)
    n_minus_1_grams_freq = generate_ngrams(tokens, n-1)
    sentence = list(prefix)
    
    while len(sentence) < length:
        seq = sentence[-(n-1):]
        next_word = predict_next_word(tokens, seq, n)
        if next_word is None:
            break
        sentence.append(next_word)
    
    return ' '.join(sentence)

In [19]:
n = 3
prefix = ['churchill', 'was']
length = 10
sentence = generate_sentence(tokens, prefix, n, length)
print(f"The generated sentence is: '{sentence}'")

The generated sentence is: 'churchill was looking on unconcerned jane was quite at loss'


## Laplace smoothing

 This is a method to handle unseen n-grams so that there is no 0 probability of any n-gram. This is done by modifying the probability calculation to account for unseen n-grams. It adds a count of 1 to every possible n-gram



In [20]:
def generate_ngrams(tokens, n):
    n_grams = ngrams(tokens, n)
    n_grams_freq = Counter(n_grams)
    return n_grams_freq

def calculate_probability_laplace(sequence, token, n_grams_freq, n_minus_1_grams_freq, vocab_size, k=1):
    seq = tuple(sequence)
    seq_freq = n_minus_1_grams_freq.get(seq, 0)
    n_gram = seq + (token,)
    n_gram_freq = n_grams_freq.get(n_gram, 0)
    
    if seq_freq == 0:
        return 0
    else:
        return (n_gram_freq + k) / (seq_freq + k * vocab_size)

''''def calculate_probability_laplace(tokens, prefix, ngram_counts, ngram_total_counts, vocabulary_size, n, k=1):
    probabilities = []
    start_index = len(prefix)
    for i in range(start_index, len(tokens)):
        ngram = tuple(tokens[i-n+1:i+1])
        ngram_count = ngram_counts[ngram]
        context_count = ngram_total_counts[ngram[:-1]]
        probability = (ngram_count + k) / (context_count + k * vocabulary_size)
        probabilities.append(probability)
    return probabilities '''

def predict_next_word_laplace(tokens, sequence, n_grams_freq, n_minus_1_grams_freq, vocab_size):
    possible_words = set(tokens)
    next_possible_word = None
    max_prob = 0
    
    for word in possible_words:
        prob = calculate_probability_laplace(sequence, word, n_grams_freq, n_minus_1_grams_freq, vocab_size,1)
        if prob > max_prob:
            max_prob = prob
            next_possible_word = word
    return next_possible_word

def generate_sentence_laplace(tokens, prefix, n, length):
    n_grams_freq = generate_ngrams(tokens, n)
    n_minus_1_grams_freq = generate_ngrams(tokens, n-1)
    vocab_size = len(set(tokens))
    
    sentence = list(prefix)
    
    while len(sentence) < length:
        seq = sentence[-(n-1):]
        next_word = predict_next_word_laplace(tokens, seq, n_grams_freq, n_minus_1_grams_freq, vocab_size)
        if next_word is None:
            break
        sentence.append(next_word)
    
    return ' '.join(sentence)

# Example usage with various (n-1)-gram sequences
n = 3
prefix = ['emma', 'saw']
length = 10
sentence = generate_sentence_laplace(tokens, prefix, n, length)
print(f"The generated sentence with Laplace is: '{sentence}'")

The generated sentence with Laplace is: 'emma saw his son in law and generally strong sense'


## Testing and evaluation

### Test the model by inputting various (n − 1)-gram sequences

In [36]:
test_sequences = [
    ['Baseball', 'is'],
    ['seasons','will','bring'],
    ['i','sing','to'],
    ['think','in','silence'],
    ['ponder','in','silence']
]

n = 3  # Using trigrams
length = 10  # Desired length of the generated sentence

for seq in test_sequences:
    sentence = generate_sentence_laplace(tokens, seq, n, length)
    print(f"Input sequence: {seq}")
    print(f"Generated sentence: '{sentence}'\n")

Input sequence: ['Baseball', 'is']
Generated sentence: 'Baseball is'

Input sequence: ['seasons', 'will', 'bring']
Generated sentence: 'seasons will bring him soon continued mrs weston was the'

Input sequence: ['i', 'sing', 'to']
Generated sentence: 'i sing to her and she was not to be'

Input sequence: ['think', 'in', 'silence']
Generated sentence: 'think in silence smiles of intelligence passed between them emma'

Input sequence: ['ponder', 'in', 'silence']
Generated sentence: 'ponder in silence smiles of intelligence passed between them emma'



## Perplexity Score

In [22]:
def calculate_probability(tokens, ngram_counts, ngram_total_counts, vocabulary_size, n, k=1):
    probabilities = []
    for i in range(n-1, len(tokens)):
        ngram = tuple(tokens[i-n+1:i+1])
        ngram_count = ngram_counts[ngram]
        context_count = ngram_total_counts[ngram[:-1]]
        probability = (ngram_count + k) / (context_count + k * vocabulary_size)
        probabilities.append(probability)
    return probabilities

In [23]:
''''def calculate_perplexity_1(tokens, n, n_grams_freq, n_minus_1_grams_freq, vocab_size):
    log_prob_sum = 0
    n_tokens = len(tokens)
    
    for i in range(n_tokens - n + 1):
        seq = tokens[i:i+n-1]
        target = tokens[i+n-1]
        prob = calculate_probability_laplace(seq, target, n_grams_freq, n_minus_1_grams_freq, vocab_size,1)
        if prob > 0:
            print('probability greater than 0: ',log_prob_sum)
            log_prob_sum += math.log(prob)
        else:
            log_prob_sum += float('-inf')  # Handle zero probability
            print('probability less than 0: ',log_prob_sum)
    
    perplexity = math.exp(-log_prob_sum / (n_tokens - n + 1))
    return perplexity '''

"'def calculate_perplexity_1(tokens, n, n_grams_freq, n_minus_1_grams_freq, vocab_size):\n    log_prob_sum = 0\n    n_tokens = len(tokens)\n    \n    for i in range(n_tokens - n + 1):\n        seq = tokens[i:i+n-1]\n        target = tokens[i+n-1]\n        prob = calculate_probability_laplace(seq, target, n_grams_freq, n_minus_1_grams_freq, vocab_size,1)\n        if prob > 0:\n            print('probability greater than 0: ',log_prob_sum)\n            log_prob_sum += math.log(prob)\n        else:\n            log_prob_sum += float('-inf')  # Handle zero probability\n            print('probability less than 0: ',log_prob_sum)\n    \n    perplexity = math.exp(-log_prob_sum / (n_tokens - n + 1))\n    return perplexity "

In [24]:
def calculate_perplexity(tokens, ngram_counts, ngram_total_counts, vocabulary_size, n, k=1):
    probabilities = calculate_probability(tokens, ngram_counts, ngram_total_counts, vocabulary_size, k)
    log_prob_sum = sum(math.log(p) for p in probabilities if p > 0)
    perplexity = math.exp(-log_prob_sum / len(probabilities))
    return perplexity

train-test split

In [25]:
train_size = int(0.75 * len(tokens))
train_tokens = tokens[:train_size]
test_tokens = tokens[train_size:]

In [26]:
n_grams_freq = generate_ngrams(train_tokens, n)
n_minus_1_grams_freq = generate_ngrams(train_tokens, n-1)
vocab_size = len(set(train_tokens))
n = 3

perplexity_score = calculate_perplexity(test_tokens, n_grams_freq, n_minus_1_grams_freq, vocab_size, n, 1)

In [27]:
perplexity_score 

6424.000000021067

In [33]:
def evaluate_ngram_model(n):
    n_grams_freq = generate_ngrams(train_tokens, n)
    n_minus_1_grams_freq = generate_ngrams(train_tokens, n-1)
    vocab_size = len(set(train_tokens))
    perplexity = calculate_perplexity(test_tokens, n_grams_freq, n_minus_1_grams_freq, vocab_size, n, n)
    return perplexity

# Evaluate bigram and trigram models
bigram_perplexity = evaluate_ngram_model(2)
trigram_perplexity = evaluate_ngram_model(3)

print(f"Bigram Perplexity: {bigram_perplexity}")
print(f"Trigram Perplexity: {trigram_perplexity}")

Bigram Perplexity: 1492.821411101309
Trigram Perplexity: 4916.402326143414


In [34]:
trigram_perplexity = evaluate_ngram_model(3)
quadigram_perplexity = evaluate_ngram_model(4)

print(f"Trigram Perplexity: {trigram_perplexity}")
print(f"Quadgram Perplexity: {quadigram_perplexity}")


Trigram Perplexity: 4916.402326143414
Quadgram Perplexity: 6214.487301508134


In [35]:
pentagram_perplexity = evaluate_ngram_model(5)
hexagram_perplexity = evaluate_ngram_model(6)

print(f"Pentagram Perplexity: {pentagram_perplexity}")
print(f"Hexgram Perplexity: {hexagram_perplexity}")


Pentagram Perplexity: 6394.184532890865
Hexgram Perplexity: 6418.13182790551


## Analysis of results

After evaluating different n-gram models, it's clear that the bigram model performs the best, with a perplexity score of around 1493. This means it predicts the test data better than models that consider more context, like trigrams or higher n-grams.

As we increase the context from trigrams to hexagrams, the perplexity score actually worsens. For instance, the trigram model has a perplexity of about 4916, and this trend continues upwards with quadgrams, pentagrams, and hexagrams reaching scores above 6418.

The main reason for this pattern is likely due to data sparsity. Higher n-grams need more data to make accurate predictions because they consider more words in their context. Without sufficient data, the models struggle to make reliable predictions, leading to higher perplexity. Essentially, the models have less information to learn from, making them less effective.

On the other hand, the bigram model strikes a good balance. It uses just enough context to make informed predictions without requiring an overwhelming amount of data. This simplicity makes it more robust, especially with the relatively smaller dataset we have from Gutenberg Whitman text.

In summary, for our specific dataset, sticking with a simpler bigram model is more effective than using more complex models that need more data to perform well. This insight underscores the importance of balancing model complexity with the amount of data available. In scenarios with richer data, more complex models might shine, but here, simplicity wins.

## Report on the performance and results

In conclusion, higher order n-grams require more data to make accurate text predictions. With the data we have, the higher n-grams are not performing well because adding more words for context while having less data on the whole makes the perplexity score worse.

For this data, the bigram model finds strikes a balance, using just enough context to make good predictions without requiring too much data. When we increase the context by using higher n-grams, we end up with less reliable probability estimates and higher perplexity.

More complex n-gram models can overfit the training data, meaning they perform well on the data they were trained on but poorly on new, unseen data.

## With UI code with PyQt5 - Please run these cells and GUI comes up prompting to enter inputs

In [ ]:
pip install PyQt5

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QTextEdit, QMessageBox

In [ ]:
class TextCompletionApp(QWidget):
    def __init__(self):
        super().__init__()

        self.initUI()
        
    def initUI(self):
        layout = QVBoxLayout()

        self.prefix_label = QLabel('Enter Prefix sentence:')
        layout.addWidget(self.prefix_label)
        
        self.prefix_input = QLineEdit(self)
        layout.addWidget(self.prefix_input)
        
        self.length_label = QLabel('Enter length of the sentence you to generate:')
        layout.addWidget(self.length_label)
        
        self.length_input = QLineEdit(self)
        layout.addWidget(self.length_input)
        
        self.generate_button = QPushButton('Generate', self)
        self.generate_button.clicked.connect(self.on_generate_click)
        layout.addWidget(self.generate_button)
        
        self.result_label = QLabel('Generated Text:')
        layout.addWidget(self.result_label)
        
        self.result_text = QTextEdit(self)
        self.result_text.setReadOnly(True)
        layout.addWidget(self.result_text)
        
        self.setLayout(layout)
        self.setWindowTitle('Text Completion using N-gram Model')
        self.show()
        
    def on_generate_click(self):
        prefix = self.prefix_input.text().strip().lower().split()
        length = self.length_input.text().strip()
        
        if not length.isdigit():
            QMessageBox.warning(self, 'Error', 'Length must be a number.')
            return
        
        length = int(length)
        
        if len(prefix) < n-1:
            QMessageBox.warning(self, 'Error', f'Prefix must be at least {n-1} words long.')
            return
        
        
        print('prefix is: ', prefix)
        generated_sentence = generate_sentence_laplace(tokens, prefix, n, length)
        self.result_text.setText(generated_sentence)

n = 3 

# Run the PyQt5 application
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = TextCompletionApp()
    sys.exit(app.exec_())